<a href="https://colab.research.google.com/github/VashionLochie/ArtGallery/blob/master/Visualizador_BigGan_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visualizador con BigGan (deep music visualizer)

Este notebook es para generar videos con BigGan (Brock et al., 2018) en particular vamos a traducir audio a vectores en el espacio latente para generar interpolaciones ente multiples clases.

(Esta es una version del repsitorio: https://github.com/msieg/deep-music-visualizer creado por msieg)


# GPU
Primer vamos a confirmar que tenemos una p100 de GPU que es la que vamos a necesitar para correr esta notebook

In [14]:
import psutil
def get_size(bytes, suffix="B"):
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f"{bytes:.2f}{unit}{suffix}"
        bytes /= factor
print("="*40, "Memory Information", "="*40)
svmem = psutil.virtual_memory()
print(f"Total: {get_size(svmem.total)}") ; print(f"Available: {get_size(svmem.available)}")
print(f"Used: {get_size(svmem.used)}") ; print(f"Percentage: {svmem.percent}%")

======================================== Memory Information ========================================
Total: 25.51GB
Available: 24.48GB
Used: 690.89MB
Percentage: 4.1%


In [2]:
! nvidia-smi

Tue Apr 20 02:23:11 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Ahora vamos a instalar el reposoitorio


In [3]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [4]:
!git clone https://github.com/msieg/deep-music-visualizer.git
!cd deep-music-visualizer

Cloning into 'deep-music-visualizer'...
remote: Enumerating objects: 312, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 312 (delta 21), reused 1 (delta 0), pack-reused 267
Receiving objects: 100% (312/312), 11.15 MiB | 23.29 MiB/s, done.
Resolving deltas: 100% (121/121), done.


In [5]:
!pip install -r /content/deep-music-visualizer/requirements.txt

     |████████████████████████████████| 31.2MB 172kB/s 
     |████████████████████████████████| 133kB 56.5MB/s 
     |████████████████████████████████| 81kB 9.6MB/s 
     |████████████████████████████████| 7.4MB 59.9MB/s 
ERROR: lucid 0.3.10 requires umap-learn, which is not installed.
ERROR: lucid 0.3.10 has requirement numpy<=1.19, but you'll have numpy 1.19.5 which is incompatible.
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
ERROR: botocore 1.20.54 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [6]:
!pip install lucid==0.3.8

     |████████████████████████████████| 71kB 6.5MB/s 
     |████████████████████████████████| 81kB 7.1MB/s 
     |████████████████████████████████| 1.2MB 17.0MB/s 
  Created wheel for lucid: filename=lucid-0.3.8-cp37-none-any.whl size=100658 sha256=b5a8c12377962ea75f574c3bd487f176ca4df3bca1abbcffb255deec8dbbce4f
  Stored in directory: /root/.cache/pip/wheels/79/f3/74/bec25a277fbfa9e4a8b72841377d5574e795b66f63d8d42154
  Created wheel for umap-learn: filename=umap_learn-0.5.1-cp37-none-any.whl size=76569 sha256=4531668b0c075ce8b83add38841385277b566a75f2072189abba2863388d096e
  Stored in directory: /root/.cache/pip/wheels/ad/df/d5/a3691296ff779f25cd1cf415a3af954b987fb53111e3392cf4
  Created wheel for pynndescent: filename=pynndescent-0.5.2-cp37-none-any.whl size=51351 sha256=5706209df6a524c6e798aa3d08008f5904ffdd78db2b91d8e3fdf50be9d09b16
  Stored in directory: /root/.cache/pip/wheels/ba/52/4e/4c28d04d144a28f89e2575fb63628df6e6d49b56c5ddd0c74e
Successfully built lucid umap-learn pynndesce

In [7]:
!pip install imgaug==0.2.5

     |████████████████████████████████| 563kB 13.9MB/s 
  Created wheel for imgaug: filename=imgaug-0.2.5-cp37-none-any.whl size=561439 sha256=d3a30cd32469c87b09e39100c819fa711494b9f823d6ad0f920b874ade4b6900
  Stored in directory: /root/.cache/pip/wheels/31/48/c8/ca3345e8582a078de94243996e148377ef66fdb845557bae0b
Successfully built imgaug
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9


In [8]:
!pip install scipy==1.2.3

     |████████████████████████████████| 24.8MB 1.6MB/s 
  Found existing installation: scipy 1.1.0
    Uninstalling scipy-1.1.0:
      Successfully uninstalled scipy-1.1.0


In [9]:
!pip install umap-learn==0.4.0

     |████████████████████████████████| 71kB 6.6MB/s 
     |████████████████████████████████| 3.9MB 16.7MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.4.0-cp37-none-any.whl size=66154 sha256=f6caadf55abe439a0041323a00f07a70e4667d74090af0ea2c3fd2448364338c
  Stored in directory: /root/.cache/pip/wheels/a8/75/bb/207d9afeeeabcd8c700c0512ec47e1999d79f19920b746fc7d
Successfully built umap-learn
  Found existing installation: umap-learn 0.5.1
    Uninstalling umap-learn-0.5.1:
      Successfully uninstalled umap-learn-0.5.1





# Vamos a ver como funcionan los comandos.



- song 

Este comando va a indicar que cancion vamos a usar, la misma tiene que estar alojada en la memoria virtual. y se indica con su nombre y el tipo de archivo ej: "beethoven.mp3"

- resolution

Nos da opciones de resolucion. La resolucion por defecto es de 512x512 (esta es la maxima resolucion) pero esta la opcion de tener una salida de 256x256 y 128x128 pixles,
indicandolas respectivamente 256 y 128.

- duration

marca la duracion del video, por defecto la duracion es la del audio. pero podemos indicar la cantidad de segundos que tendra el video, siendo el maximo el tiempo de duracion del audio.
esto lo podemos indicar en segundos.


- pitch_sensitivity

determina que tan rapido el vector de clase (contenido tematico del video) reacciona los cambios de tono en el audio. Mientras mas grande es el valor, mayor la sensibilidad.

Rango: 1 - 299

Recomendado:  200 – 295

Por defecto: 220


- tempo_sensitivity

determina que tan rapido el vector de noise (el que da las caracteristicas posicionales y de forma dentro de las clases)
va a reaccionar a los cambion de volumen y tempo. Mientras mas grande es el valor, mayor la sensibilidad.

 Rango recomendado:  0.05 – 0.8

 Por defecto: 0.25


 - depth
 
Determina la "Profundidad" de las imagenes.
 Los numeros cercanos a 
1 tienden a tener contenido mas variado dentro de la misma clase generando espacios y algunas imprectibilidades.
 los contenidos mas cercanos a 0 se acercan mas a estructuras complejas (estructras faciales o de perros,etc.) siempre dependiendo en la clase.

Rango: 0.01 - 1

Por defecto: 1


- num_classes

este comando va a indicar el numero de clases que vamos a usar, por defecto es 12 correspondiente a la escala tonal (A, A#, B, etc.).
si usamos num_classes nuestro numero va a ser menor al maximo 12
entonces las clases se van a asignar a los tonos con mayor precencia en el audio.

- classes

si queremos visualizar clases especificas aca tenemos un indice para buscarlas (https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a)
el numero de clases debe ser igual al de num_classes o 12, si no usamos num_classes.

Por defecto: aleatorio entre 0-999


- sort_classes_by_power

a este comando le pueden asignar 1 si quieren que las doce clases respeten el orden tonal harmonico, osea que si mi primer clase es por ejemplo 345 esta va estar asignada a A, la segunda a A#, la tercera a B, etc.

- jitter

jitter previene que los mismos vectores se repitan en musica repetitiva para que la salida de video sea mas variada. si por el contrario queremos que se repitan los mismos vectores podemos asignarle el valor 0.

Rango: 0 - 1

Por defecto:0.5

- truncation

controla el tipo de imagen que se genera segun la clase. valores cercanos a 0 dan imagenes mas reconocibles y valores mas cercanos a 1 dan lo opuesto.


Rango: 0.1 - 1

Por defecto: 1

- batch_size

esto se ultiliza para videos muy largos o muy demandantes de GPU para alivianar el proceso, reduce el procesamiento de bigGan.
Bajar a menos de 30 solo si es necesario.

Por defecto: 30

- output

este comando lo utilizamos para nombrar a nuestro archivo de salida lo hacemos con el nombre y el tipo de archivo por ej:"interpolacion larga.mp4"

Por defecto: "output.mp4"

































In [10]:
!python /content/deep-music-visualizer/visualize.py --song "/content/deep-music-visualizer/beethoven.mp3" --pitch_sensitivity 280 --tempo_sensitivity 0.2 --num_classes 3 --classes 300 456 780 --output "beethoven.mp4" 

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)3981312/45929032 bytes (8.7%)8085504/45929032 bytes (17.6%)12140544/45929032 bytes (26.4%)16318464/45929032 bytes (35.5%)20463616/45929032 bytes (44.6%)24690688/45929032 bytes (53.8%)28893184/45929032 bytes (62.9%)33120256/45929032 bytes (72.1%)37453824/45929032 bytes (81.5%)41664512/45929032 bytes (90.7%)45891584/45929032 bytes (99.9%)45929032/45929032 bytes (100.0%)
  Done
File saved as /root

# El archivo de salida
Podemos acceder al output desde el indice de archivos de la memoria virtual de colab y descargarlo haciendo click a nuestra computadora o tranferirlo a nuestro drive desde el indice de archivos haciendo drag and drop en la carpeta de my drive o drive despues de correr la celda de autorizacion acontinuacion.

Recuerden que la memoria virtual se resetea y si no bajamos o pasamos el material al drive se *ELIMINA*